# Checkpoints 
### 1）啥
#### 用来保存训练模型的节点，保存正在训练模型的各个参数
### 2）为啥
#### 因为跑一个深度学习模型真的要很久啊，动不动几十个小时，假设你已经跑十个小时了，突然断了，那么：
#### 有checkpoints ------》捡起参数继续跑
#### 没有checkpoints/SaveModel ------》再来十个小时
#### 此外，对于需要抢占式使用GPU资源的同学们，在运行模型过程中不断保存参数是非常必要的 -v-
## 所以 开始咯

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

以上 from _future_ 这部分在<a>第一个tutorial</a>中有提到过。
<BR> 这一部分主要侧重于Checkpoints，一些细节可能会略过，抱歉

In [2]:
import tensorflow as tf
import iris_data

In [3]:
(train_x, train_y), (test_x, test_y) = iris_data.load_data()

In [4]:
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [14]:
# Build a DNN with 2 hidden layers and 10 nodes in each hidden layer.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3,
    model_dir = 'lalala/iris')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'lalala/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11e3991d0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# * 注意model_dir，该参数是用来存储Checkpoints的目录

In [15]:
batch_size = 100
train_steps = 1000

In [16]:
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, batch_size),
    steps=train_steps)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from lalala/iris/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into lalala/iris/model.ckpt.
INFO:tensorflow:loss = 4.998205, step = 1001
INFO:tensorflow:global_step/sec: 454.473
INFO:tensorflow:loss = 3.3090506, step = 1101 (0.223 sec)
INFO:tensorflow:global_step/sec: 531.457
INFO:tensorflow:loss = 3.2085676, step = 1201 (0.187 sec)
INFO:tensorflow:global_step/sec: 528.885
INFO:tensorflow:loss = 5.372916, step = 1301 (0.189 sec)
INFO:tensorflow:global_step/sec: 575.222
INFO:tensorflow:loss = 5.0610714, step = 1401 (0.174 sec)
INFO:tensorflow:global_step/sec: 552.78
INFO:tensorflow:loss = 4.715623, step = 1501 (0.181 sec)
INFO:tensorflow:global_step/sec: 469.365
INFO:tensorflow:loss = 4.8760653, step = 1601 (0.215 sec)
INFO:tensorflow:global_step/sec: 409.088
INFO:tensorflow:loss = 2.9384139, step = 1701 (0.243 sec)
INFO:tensorflow:global_step/sec: 550.522
INFO:tensorflow:loss = 5.4623785, ste

### 以上信息告诉我们Checkpoints的保存位置，及所保存的文件名称
Create CheckpointSaverHook ... Restoring parameters from lalala/iris/model.ckpt-1000 ...Saving checkpoints for 1001 into lalala/iris/model.ckpt
### 也就是，一旦开始Estimator开始训练，就会自动在硬盘写入如下内容：
#### 1）Checkpoints ：包括训练过程模型的各种信息
#### 2）Eventfiles：包含后期可以用 <a> TensorBoard </a>可视化的信息

就是这样婶儿的
<br>
<img src="https://www.tensorflow.org/images/first_train_calls.png?hl=zh-cn">

### 不过，
#### 你也许有注意到，在<a>小白样例</a>中我们并没有在tf.estimator.DNNClassifier()中加入model_dir这个参数
#### 就是像下面这样👇

In [17]:
# Build a DNN with 2 hidden layers and 10 nodes in each hidden layer.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/8l/j0ddzz_x75n85vq8tp86dwpr0000gn/T/tmp44v79l2a', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11f927b38>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## 那此时Checkpoints存储在哪里呢 (⊙v⊙)

In [19]:
print(classifier.model_dir)

/var/folders/8l/j0ddzz_x75n85vq8tp86dwpr0000gn/T/tmp44v79l2a


### 就是这里了 👆
# -------------------------------------------------------------------------------------------

## 关于Checkpoints本身的参数设置
### 1）默认情况下的Checkpoints
* 每隔十分钟写一次Checkpoints （也就是你最多浪费十分钟再训练时间的意思
* 更具体来说，在训练过程中第一次迭代开始后以及最后一次迭代之间写入Checkpoints
* 在目录中仅保存五次最近的Checkpoints

### 2）假设你对以上默认设置并不满意，你可以：
* 创建一个<a>RunConfig</a>对象，定义专属Checkpoints
* 当初始化Estimator时，将RunConfig对象传入Estimator的config参数
### 例如，我想每秒保存一次Checkpoints，保存最近的10个Checkpoints

In [23]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 1,  # 注意是以秒作为单位哦
    keep_checkpoint_max = 10,       # 保存最后十次运行节点
)

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3,
    model_dir='www/iris',
    config=my_checkpointing_config) #嗯是这里的config

INFO:tensorflow:Using config: {'_model_dir': 'www/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1, '_session_config': None, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11eb4e2e8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, batch_size),
    steps=train_steps)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from www/iris/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into www/iris/model.ckpt.
INFO:tensorflow:loss = 3.0924716, step = 1001
INFO:tensorflow:global_step/sec: 425.63
INFO:tensorflow:loss = 5.1215634, step = 1101 (0.245 sec)
INFO:tensorflow:global_step/sec: 373.101
INFO:tensorflow:loss = 5.199152, step = 1201 (0.262 sec)
INFO:tensorflow:Saving checkpoints for 1226 into www/iris/model.ckpt.
INFO:tensorflow:global_step/sec: 148.014
INFO:tensorflow:loss = 4.1368327, step = 1301 (0.674 sec)
INFO:tensorflow:global_step/sec: 653.271
INFO:tensorflow:loss = 6.0438037, step = 1401 (0.153 sec)
INFO:tensorflow:global_step/sec: 516.938
INFO:tensorflow:loss = 4.880019, step = 1501 (0.193 sec)
INFO:tensorflow:Saving checkpoints for 1520 into www/iris/model.ckpt.
INFO:tensorflow:global_step/sec: 140.736
INFO:tensorflow:loss = 4.869627, step = 1601 (0.712 sec)
INFO:tensorflow:global_step/sec: 518.51
INF

### 有没有发现Checkpoints变得多了起来， 哈。
# ----------------------------------------------------------------------------------------

## 重建模型 
（从哪里跌倒 就从哪里爬起来

### 每当调用train(),evaluate(),predict()方法时，TensorFlow将自动从Checkpoints中读取信息重建模型
#### 就是下面这样的 👇

<img src="https://www.tensorflow.org/images/subsequent_calls.png?hl=zh-cn">

### 一点点温馨提示
### 1）重建的时候记得不要改参数，例如隐藏层数量啦，Checkpoints目录啦
### 2）以上仅限于Estimator方法构建的神经网络QAQ